In [ ]:
%pip install sentence-transformers pandas tqdm

import pandas as pd
import torch
import numpy as np
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# ✅ 1. 모델 로드 (한국어 SBERT)
import torch
from sentence_transformers import SentenceTransformer
import pandas as pd

if torch.backends.mps.is_available():
    device = 'mps'
    print("🍎 Mac GPU (MPS)")
elif torch.cuda.is_available():
    device = 'cuda'
    print("🚀 NVIDIA GPU (CUDA)")
else:
    device = 'cpu'
    print("💻 CPU")

model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

try:
    model = model.to(device)
    print(f"✅ 모델을 {device}로 이동완료")
except Exception as e:
    print(f"⚠️ {device} 오류: {e}")
    device = 'cpu'
    model = model.to(device)
    print("🔄 CPU로 fallback")

# 새로운 데이터 파일 불러오기
original_df = pd.read_csv("crawling_origin.csv")
summary_df = pd.read_csv("news_summaries.csv")

print(f"원본 전문: {original_df.shape}")
print(f"KoBERT 요약: {summary_df.shape}")

# 데이터 컬럼 확인
print(f"\n원본 컬럼: {original_df.columns.tolist()}")
print(f"요약 컬럼: {summary_df.columns.tolist()}")


🍎 Mac GPU (MPS)를 사용합니다
선택된 디바이스: mps
✅ 모델을 mps로 이동완료
merged_df 크기: (10952, 9)
original_df 크기: (10952, 19)


In [ ]:
# 데이터 병합 (공통 식별자 기준)
# 컬럼명을 확인한 후 적절한 key로 merge (예: 'id', 'news_id', 'index' 등)

# 임시로 인덱스 기준 병합 (실제 컬럼명에 따라 수정 필요)
if len(original_df) == len(summary_df):
    # 길이가 같으면 인덱스 기준으로 병합
    data = pd.concat([original_df.reset_index(drop=True), summary_df.reset_index(drop=True)], axis=1)
    print("✅ 인덱스 기준 병합 완료")
else:
    # 공통 컬럼이 있다면 해당 컬럼으로 merge
    # 예시: data = pd.merge(original_df, summary_df, on='news_id', how='inner')
    print("⚠️ 데이터 길이가 다릅니다. 공통 식별자로 병합이 필요합니다.")
    print("컬럼을 확인한 후 적절한 merge key를 설정하세요.")

# 원본 전문과 요약문 컬럼 추출 (실제 컬럼명에 맞게 수정)
# 예상 컬럼명들: 'content', 'article', 'text', 'summary', 'kobert_summary' 등
try:
    # 원본 전문 컬럼 찾기
    original_cols = [col for col in original_df.columns if any(keyword in col.lower() 
                    for keyword in ['content', 'article', 'text', '본문', '전문'])]
    
    # 요약문 컬럼 찾기  
    summary_cols = [col for col in summary_df.columns if any(keyword in col.lower() 
                   for keyword in ['summary', '요약', 'kobert'])]
    
    if original_cols and summary_cols:
        originals = data[original_cols[0]].astype(str).tolist()
        summaries = data[summary_cols[0]].astype(str).tolist()
        print(f"✅ 원본: {original_cols[0]}, 요약: {summary_cols[0]}")
    else:
        print("❌ 컬럼을 찾을 수 없습니다. 수동으로 설정하세요:")
        print("원본 컬럼들:", original_df.columns.tolist())
        print("요약 컬럼들:", summary_df.columns.tolist())
        
except Exception as e:
    print(f"❌ 컬럼 처리 오류: {e}")
    print("수동으로 컬럼명을 지정하세요:")
    # originals = data['컬럼명'].astype(str).tolist()
    # summaries = data['컬럼명'].astype(str).tolist()

print(f"\n병합된 데이터: {data.shape}")
print(f"원본: {len(originals)}, 요약: {len(summaries)}")
print(f"\n원본 예시: {originals[0][:200]}...")
print(f"KoBERT 요약 예시: {summaries[0][:200]}...")


병합된 데이터 크기: (271748, 27)
요약문 개수: 271748, 원문 개수: 271748

첫 번째 요약문 예시:
이재명 대통령은 경기도 가평과 충남 서산·예산, 전남 담양, 경남 산청·합천 등 6개 시군을 특별재난지역으로 선포했는데, 엄혹한 현장에서 음주가무를 즐기는 공직자들의 처신 문제도 거론했다.

첫 번째 원문 예시:
[앵커]

 이재명 대통령이 경기도 가평과 전남 담양, 경남 산청 등 6개 시군을 특별재난지역으로 선포했습니다.

 이 대통령은 재난 상황에서 부적절한 공직자들의 처신 문제도 거론했는데, 엄혹한 현장에서 음주가무를 즐기는 정신 나간 경우도 있었다며 엄히 단속하라고 주문했습니다.

 보도에 이희연 기자입니다.

 [리포트]

경기도 가평과 충남 서산 예산..


In [ ]:
def get_memory_info():
    if device == 'mps':
        return "🍎 MPS 사용 중"
    elif device == 'cuda':
        try:
            allocated = torch.cuda.memory_allocated() / 1024**3
            cached = torch.cuda.memory_reserved() / 1024**3
            return f"🚀 CUDA - 할당: {allocated:.2f}GB, 캐시: {cached:.2f}GB"
        except:
            return "🚀 CUDA 메모리 정보 불가"
    else:
        return "💻 CPU 모드"

def encode_with_progress(model, texts, batch_size=32, description="Encoding"):
    embeddings = []
    print(f"시작: {get_memory_info()}")
    
    for i in tqdm(range(0, len(texts), batch_size), desc=description):
        batch_texts = texts[i:i+batch_size]
        batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
        embeddings.append(batch_embeddings)
        
        if i % (batch_size * 10) == 0:
            if device == 'cuda':
                torch.cuda.empty_cache()
            elif device == 'mps':
                torch.mps.empty_cache()
    
    result = torch.cat(embeddings, dim=0)
    print(f"완료: {get_memory_info()}")
    return result

print("🔄 KoBERT 요약 성능 평가 시작...")
print(f"시작점: {get_memory_info()}")

print("\n1. 원본 전문 임베딩...")
original_emb = encode_with_progress(model, originals, batch_size=32, description="원본 전문")

print("\n2. KoBERT 요약 임베딩...")
summary_emb = encode_with_progress(model, summaries, batch_size=32, description="KoBERT 요약")

print("\n3. 의미 유사도 계산...")
similarities = util.cos_sim(summary_emb, original_emb).diagonal()
score = similarities.mean().item()

print(f"\n✅ KoBERT 요약 성능 평가 결과:")
print(f"📊 평균 의미 유사도: {score:.4f}")
print(f"📈 최고 유사도: {similarities.max().item():.4f}")
print(f"📉 최저 유사도: {similarities.min().item():.4f}")
print(f"📏 표준편차: {similarities.std().item():.4f}")

# 성능 등급 분류
if score >= 0.8:
    grade = "🏆 우수"
elif score >= 0.7:
    grade = "👍 양호"
elif score >= 0.6:
    grade = "⚠️ 보통"
else:
    grade = "❌ 개선필요"

print(f"🎯 KoBERT 요약 성능: {grade} ({score:.4f})")

# 결과를 데이터에 추가
similarity_scores = similarities.cpu().numpy()

if USE_SAMPLE:
    sample_data['KoBERT_Similarity'] = similarity_scores
    print(f"\n📋 상위 5개 기사 유사도:")
    print(sample_data[['KoBERT_Similarity']].head())
    
    # 우수한 요약 예시 (상위 3개)
    top_indices = np.argsort(similarity_scores)[-3:]
    print(f"\n🏆 우수한 요약 예시 (유사도 상위 3개):")
    for i, idx in enumerate(top_indices[::-1]):
        print(f"\n--- {i+1}위 (유사도: {similarity_scores[idx]:.4f}) ---")
        print(f"원본: {originals[idx][:150]}...")
        print(f"요약: {summaries[idx][:100]}...")

print(f"\n최종: {get_memory_info()}")


🔄 임베딩 생성 시작...
시작 시점: 🍎 MPS 사용 중 (메모리 정보 제한적)

1. 요약문 임베딩 중...
시작 전: 🍎 MPS 사용 중 (메모리 정보 제한적)


요약문:  41%|████      | 3457/8493 [11:46<17:08,  4.89it/s]


KeyboardInterrupt: 

In [ ]:
# ✅ 4. 샘플링 옵션 (빠른 테스트용)
USE_SAMPLE = True  # 빠른 테스트를 위해 True로 설정
SAMPLE_SIZE = 1000

if USE_SAMPLE:
    import numpy as np
    sample_indices = np.random.choice(len(data), min(SAMPLE_SIZE, len(data)), replace=False)
    sample_data = data.iloc[sample_indices].copy()
    summaries = sample_data['요약문'].astype(str).tolist()
    originals = sample_data['본문'].astype(str).tolist()
    print(f"🚀 샘플 모드: {len(summaries)}개 데이터로 테스트")
else:
    print(f"📊 전체 데이터 모드: {len(summaries)}개 데이터로 평가")

# 현재 설정 확인
print(f"\n🔧 디바이스 정보:")
print(f"- 모델 디바이스: {model.device}")
print(f"- 현재 사용 디바이스: {device}")

# Mac MPS 정보
if torch.backends.mps.is_available():
    print("🍎 Mac MPS (Metal Performance Shaders) 지원")
    print(f"- MPS 사용 가능: {torch.backends.mps.is_built()}")
else:
    print("❌ MPS 지원 안됨")

# CUDA 정보 (Windows/Linux)
if torch.cuda.is_available():
    try:
        device_count = torch.cuda.device_count()
        current_device = torch.cuda.current_device()
        device_name = torch.cuda.get_device_name(current_device)
        
        print(f"🚀 CUDA 정보:")
        print(f"- GPU 개수: {device_count}")
        print(f"- 현재 GPU: {device_name}")
        
        total_memory = torch.cuda.get_device_properties(current_device).total_memory
        allocated = torch.cuda.memory_allocated(current_device)
        cached = torch.cuda.memory_reserved(current_device)
        
        print(f"- 총 GPU 메모리: {total_memory / 1024**3:.1f}GB")
        print(f"- 할당된 메모리: {allocated / 1024**3:.2f}GB")
        print(f"- 캐시된 메모리: {cached / 1024**3:.2f}GB")
        
    except Exception as e:
        print(f"- CUDA 정보 조회 중 오류: {e}")

# CPU 정보
if device == 'cpu':
    print("💻 CPU 모드로 실행됩니다")
    
print(f"\n📊 처리할 데이터 개수: {len(summaries)}")

NameError: name 'summaries' is not defined